# Trajectory Analysis With MDanlysis
+ Dr. Alexis Salas Burgos
+ Nanocell Labs
+ Universidad de Concepción
+ June 2019

## MDAnalysis 
* from https://www.mdanalysis.org
* conda install mdanalysis -c conda-forge
* conda install MDAnalysisTests -c conda-forge

In [2]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PSF, DCD

## MDAnalysis
u = mda.Universe(PSF, DCD)

print(">>>MDAnalysis: Universe")
print(u)

>>>MDAnalysis: Universe
<Universe with 3341 atoms>


## Subset from topology

In [3]:
## MDAnalysis subset for topology
atomos   = list(u.atoms[:3])
residuos = list(u.atoms[0:100].residues)
segids   = list(u.atoms.segments)

resnames = u.atoms[0:100].residues.resnames
resids   = u.atoms[0:100].residues.resids
res_list = list(zip(resnames, resids))

import pprint
print(">>Objetos Aminoácidos")
pprint.pprint(atomos)
print(">>Objetos Residuos")
pprint.pprint(residuos)
print(">>Objetos Segmentos")
pprint.pprint(segids)

print(">>Lista de Residuos")
pprint.pprint(res_list)

>>Objetos Aminoácidos
[<Atom 1: N of type 56 of resname MET, resid 1 and segid 4AKE>,
 <Atom 2: HT1 of type 2 of resname MET, resid 1 and segid 4AKE>,
 <Atom 3: HT2 of type 2 of resname MET, resid 1 and segid 4AKE>]
>>Objetos Residuos
[<Residue MET, 1>,
 <Residue ARG, 2>,
 <Residue ILE, 3>,
 <Residue ILE, 4>,
 <Residue LEU, 5>]
>>Objetos Segmentos
[<Segment 4AKE>]
>>Lista de Residuos
[('MET', 1), ('ARG', 2), ('ILE', 3), ('ILE', 4), ('LEU', 5)]


# RMSD

In [4]:
## MDAnalysis
# project a dynamic property on the structure using the B-factor (tempfactor) field
import numpy as np
import MDAnalysis
import MDAnalysis.analysis.align

from MDAnalysis.tests.datafiles import PSF, DCD


u = MDAnalysis.Universe(PSF, DCD)
ref = MDAnalysis.Universe(PSF, DCD)  # copy of u

CORE_selection = "resid 1-29 or resid 60-121 or resid 160-214"
pdbtrj = "adk_distance_bfac.pdb"


# workaround for Issue #1359 https://github.com/MDAnalysis/mdanalysis/issues/1359
# which will be fixed in 0.17.0
u.add_TopologyAttr(MDAnalysis.core.topologyattrs.Tempfactors(np.zeros(len(u.atoms))))


with MDAnalysis.Writer(pdbtrj, multiframe=True, bonds=None, n_atoms=u.atoms.n_atoms) as PDB:
    # reference coordinates: set to first frame
    ref.trajectory[0]
    # iterate through our trajectory
    for ts in u.trajectory:
        # superimpose on the reference CORE (at t=0)
        rmsd = MDAnalysis.analysis.align.alignto(u.atoms, ref.atoms, select=CORE_selection)
        distances = np.sqrt(np.sum((u.atoms.positions - ref.atoms.positions)**2, axis=1))
        # project displacement on structure via bfactor ("tempfactor") field
        u.atoms.tempfactors = distances
        PDB.write(u.atoms)
        print("Frame {0}: CORE RMSD before/after superposition: {1[0]:.1f} / {1[1]:.1f} A. "
              "min-max displacement: {2:.1f}...{3:.1f} A".format(ts.frame, rmsd, distances.min(), distances.max()))

print("Wrote PDB trajectory {0} with distances in bfactor field".format(pdbtrj))

/usr/local/anaconda3/envs/amber2019/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  "".format(attrname, default))
/usr/local/anaconda3/envs/amber2019/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  "".format(attrname, default))
/usr/local/anaconda3/envs/amber2019/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  "".format(attrname, default))


Frame 0: CORE RMSD before/after superposition: 0.0 / 0.0 A. min-max displacement: 0.0...0.0 A
Frame 1: CORE RMSD before/after superposition: 0.7 / 0.7 A. min-max displacement: 0.0...3.5 A
Frame 2: CORE RMSD before/after superposition: 0.8 / 0.8 A. min-max displacement: 0.0...3.6 A
Frame 3: CORE RMSD before/after superposition: 1.0 / 1.0 A. min-max displacement: 0.0...4.2 A
Frame 4: CORE RMSD before/after superposition: 1.1 / 1.0 A. min-max displacement: 0.0...4.7 A
Frame 5: CORE RMSD before/after superposition: 1.1 / 1.1 A. min-max displacement: 0.0...6.3 A
Frame 6: CORE RMSD before/after superposition: 1.2 / 1.1 A. min-max displacement: 0.1...6.4 A
Frame 7: CORE RMSD before/after superposition: 1.3 / 1.2 A. min-max displacement: 0.1...7.2 A
Frame 8: CORE RMSD before/after superposition: 1.3 / 1.2 A. min-max displacement: 0.1...7.3 A
Frame 9: CORE RMSD before/after superposition: 1.4 / 1.3 A. min-max displacement: 0.0...6.6 A
Frame 10: CORE RMSD before/after superposition: 1.5 / 1.3 A.

## Radius Girus

In [10]:
Rgyr = []
protein = u.select_atoms("protein")
for ts in u.trajectory:
    Rgyr.append((u.trajectory.time, protein.radiusofgyration()))
Rgyr = np.array(Rgyr)

AttributeError: AtomGroup has no attribute radiusofgyration

In [11]:
# quick plot
import matplotlib.pyplot as plt
ax = plt.subplot(111)
ax.plot(Rgyr[:,0], Rgyr[:,1], 'r--', lw=2, label=r"$R_G$")
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"radius of gyration $R_G$ ($\AA$)")
ax.figure.savefig("Rgyr.pdf")
plt.draw()

TypeError: list indices must be integers or slices, not tuple

# Hydrogen Bond

In [ ]:
## MDAnalysis
import MDAnalysis
import MDAnalysis.analysis.hbonds
h = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 'protein', 'protein', distance=3.0, angle=120.0)
h.run()

In [ ]:
h.generate_table()
import pandas as pd
df = pd.DataFrame.from_records(h.table)
df.hist(column=["angle", "distance"])